In [96]:
import pandas as pd
import pandas as pd
import plotly.graph_objects as go
import plotly.io as pio

bridges_path = r"C:\Users\relia\Documents\GitHub\Bridges\Data\df_bridges_2024_all.csv"

df_bridges_all = pd.read_csv(bridges_path, encoding='utf-8', delimiter=',')

In [97]:
# Assuming df_bridges_all is your DataFrame
columns_to_convert = ['Zemepisná_dĺžka', 'Zemepisná_šírka', 'Výška', 'Normálna_zaťažiteľnosť',
                      'Výhradná_zaťažiteľnosť_t', 'Výnimočná_zaťažiteľnosť_t', 'Počet_otvorov',
                      'Dĺžka_nosnej_konštrukcie_m', 'Dĺžka_premostenia', 'Volná_šírka_mosta',
                      'Šírka_medzi_obrubami_m', 'Plocha_mosnej_konštrukcie_m2', 'Plocha_mosta_m2']

# Convert specified columns to float
df_bridges_all[columns_to_convert] = df_bridges_all[columns_to_convert].apply(
    pd.to_numeric, errors='coerce')

# Convert Rok_postavenia to integer, assign NaN if not possible
df_bridges_all['Rok_postavenia'] = pd.to_numeric(
    df_bridges_all['Rok_postavenia'], errors='coerce')

df_bridges_all = df_bridges_all[df_bridges_all["n_2024"] != 0]

df_bridges_all.columns

Index(['ID_mosta', 'Trieda_PK', 'Číslo_PK', 'Názov_mosta', 'správcovské_číslo',
       'Zemepisná_šírka', 'Zemepisná_dĺžka', 'Výška', 'Druh_konštrukcie',
       'Material', 'Normálna_zaťažiteľnosť', 'Výhradná_zaťažiteľnosť_t',
       'Výnimočná_zaťažiteľnosť_t', 'Spôsob_stanovenia', 'Predmet_premostenia',
       'Rok_postavenia', 'Zaťažovacia_trieda_STN', 'Počet_otvorov',
       'Dĺžka_premostenia', 'Dĺžka_nosnej_konštrukcie_m', 'Volná_šírka_mosta',
       'Šírka_medzi_obrubami_m', 'Plocha_mosnej_konštrukcie_m2',
       'Plocha_mosta_m2', 'Vlastník_úseku', 'Správca_úseku', 'Okres',
       'Počiatočný_uzol', 'Koncový_uzol', 'úsek_ID', 'Kraj', 'n_2012',
       'n_2013', 'n_2014', 'n_2015', 'n_2016', 'n_2017', 'n_2018', 'n_2019',
       'n_2020', 'n_2021', 'n_2022', 'n_2023', 'n_2024'],
      dtype='object')

In [98]:
df_bridges_all["Material"].value_counts()

Material
monolitický železobetón          3264
prefabrikovaný železobetón       1917
prefabrikovaný predpätý betón    1658
monolitický predpätý betón        359
oceľ -  plnostenná                268
monolitický prostý betón          229
kameň                             193
Nezistený                         105
spriahnutá betón-betón             77
spriahnutá oceľ-betón              31
oceľobetón                         27
tehla                              26
iný materiál                       22
oceľ -  priehradová                18
Name: count, dtype: int64

In [99]:
filtered_bridges = df_bridges_all[
    (df_bridges_all['Správca_úseku'] == "SC BSK") &
    (df_bridges_all["Material"] == 'prefabrikovaný predpätý betón')
]

In [100]:
filtered_bridges[["ID_mosta", "n_2024"]]

,ID_mosta,n_2024
1842,M2798,4
1844,M1734,4
1861,M221,5
1893,M2684,5
1894,M7488,4
1906,M5538,1
1908,M2907,3
1914,M2273,5
1918,M6631,6
3131,M1016,5


2095    M2798
3199    M1734
17428     M221
10883    M2684
7592    M7488
11201    M5538
531     M5538_AB
3100    M5538_AC
1970    M5538_AD
433     M5538_BA
4040    M5538_CA
2037    M5538_DA
14997    M2907 -> rozdeliť
9840    M2907_AB
3236    M2907_AC
NaN    M2273
5407    M6631
3415    M1016
2601    M3641
2579    M3844
NaN    M7647
2746    M6067
1939    M4572
NaN    M2485
NaN    M4874
3479    M4006
5152    M1108
2141    M1702
4107    M7109
NaN    M4912

In [101]:
def calculate_relative_condition_counts(data, start_year=2012, end_year=2024):
    relative_counts = pd.DataFrame()

    for year in range(start_year, end_year + 1):
        column_name = f"n_{year}"
        year_data = data[data[column_name] > 0]  # Exclude bridges not assessed
        counts = year_data[column_name].value_counts().rename_axis(
            'Condition').reset_index(name=column_name)
        total = counts[column_name].sum()
        counts[column_name] = (counts[column_name] / total) * 100

        if relative_counts.empty:
            relative_counts = counts
        else:
            relative_counts = pd.merge(
                relative_counts, counts, on='Condition', how='outer')

    relative_counts.fillna(0, inplace=True)
    relative_counts.sort_values('Condition', inplace=True)
    return relative_counts


def create_plot(relative_counts, title, filename):
    color_palette = ['#46a7a2', '#e7a564', '#8bae7c',
                     '#353a29', '#eae1a9', '#c25469', '#2d505d']
    data_for_plotting = relative_counts.set_index('Condition').T.reset_index()
    data_for_plotting.rename(columns={'index': 'Year'}, inplace=True)
    years = ['{}'.format(year) for year in range(2012, 2025)]
    data_for_plotting['Year'] = years

    fig = go.Figure()

    for i, condition in reversed(list(enumerate(relative_counts['Condition'].unique()))):
        fig.add_trace(go.Scatter(
            x=data_for_plotting['Year'],
            y=data_for_plotting[condition],
            mode='lines',
            stackgroup='one',
            name=str(condition),
            line=dict(color=color_palette[i % len(color_palette)])
        ))

    fig.update_layout(
        title=title,
        yaxis_title='percentuálne zastúpenie',
        yaxis=dict(ticksuffix="%", range=[0, 100]),
        font=dict(family="Calibri, sans-serif", size=14),
        legend=dict(
            orientation="h",
            yanchor="top",
            y=1.080,
            xanchor="right",
            x=1
        ),
        xaxis=dict(
            tickvals=[str(year) for year in range(2012, 2025)],
            ticktext=["'12", "'13", "'14", "'15", "'16", "'17",
                      "'18", "'19", "'20", "'21", "'22", "'23", "'24"]
        ),
        plot_bgcolor='#EEEEEE',
        margin=dict(l=0, r=20, t=75, b=25),
        width=750,
        height=500
    )

    fig.show()

In [102]:
# Calculate relative condition counts for each category
relative_condition_counts = calculate_relative_condition_counts(
    filtered_bridges)

In [103]:
filtered_bridges.columns

Index(['ID_mosta', 'Trieda_PK', 'Číslo_PK', 'Názov_mosta', 'správcovské_číslo',
       'Zemepisná_šírka', 'Zemepisná_dĺžka', 'Výška', 'Druh_konštrukcie',
       'Material', 'Normálna_zaťažiteľnosť', 'Výhradná_zaťažiteľnosť_t',
       'Výnimočná_zaťažiteľnosť_t', 'Spôsob_stanovenia', 'Predmet_premostenia',
       'Rok_postavenia', 'Zaťažovacia_trieda_STN', 'Počet_otvorov',
       'Dĺžka_premostenia', 'Dĺžka_nosnej_konštrukcie_m', 'Volná_šírka_mosta',
       'Šírka_medzi_obrubami_m', 'Plocha_mosnej_konštrukcie_m2',
       'Plocha_mosta_m2', 'Vlastník_úseku', 'Správca_úseku', 'Okres',
       'Počiatočný_uzol', 'Koncový_uzol', 'úsek_ID', 'Kraj', 'n_2012',
       'n_2013', 'n_2014', 'n_2015', 'n_2016', 'n_2017', 'n_2018', 'n_2019',
       'n_2020', 'n_2021', 'n_2022', 'n_2023', 'n_2024'],
      dtype='object')

In [104]:
relative_condition_counts

,Condition,n_2012,n_2013,n_2014,n_2015,n_2016,n_2017,n_2018,n_2019,n_2020,n_2021,n_2022,n_2023,n_2024
0,1,4.545455,4.545455,4.545455,9.090909,9.090909,4.545455,4.545455,4.545455,4.545455,4.545455,4.545455,4.545455,4.545455
1,2,4.545455,4.545455,4.545455,9.090909,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,3,36.363636,36.363636,36.363636,18.181818,13.636364,13.636364,13.636364,13.636364,9.090909,4.545455,4.545455,4.545455,4.545455
3,4,54.545455,50.000000,50.000000,50.000000,45.454545,45.454545,54.545455,54.545455,45.454545,40.909091,40.909091,40.909091,40.909091
4,5,0.000000,4.545455,4.545455,4.545455,18.181818,22.727273,18.181818,13.636364,18.181818,27.272727,27.272727,27.272727,27.272727
5,6,0.000000,0.000000,0.000000,9.090909,13.636364,13.636364,9.090909,13.636364,22.727273,22.727273,22.727273,22.727273,22.727273


In [105]:
# Create and save plots for each category
create_plot(relative_condition_counts,
            'Vývoj STS predpätých prefabrikovaných mostov v správe SC BSK', 'Vyvoj_I')

In [106]:
precast_postensioned = filtered_bridges[[
    "ID_mosta", "Názov_mosta", 'Rok_postavenia', 'Dĺžka_premostenia', 'Zemepisná_šírka', 'Zemepisná_dĺžka', 'n_2012',
    'n_2013', 'n_2014', 'n_2015', 'n_2016', 'n_2017', 'n_2018', 'n_2019',
    'n_2020', 'n_2021', 'n_2022', 'n_2023', 'n_2024']].copy()
precast_postensioned.reset_index(drop=True, inplace=True)

In [107]:
import pandas as pd
import numpy as np

# Assuming precast_postensioned is your original DataFrame
# Define the traffic intensities and bridge IDs
traffic_data = {
    'ID_mosta': [
        'M2798', 'M1734', 'M221', 'M2684', 'M7488', 'M5538', 'M2907', 'M2273', 'M6631',
        'M1016', 'M3641', 'M3844', 'M7647', 'M6067', 'M4572', 'M2485', 'M4874', 'M4006',
        'M1108', 'M1702', 'M7109', 'M4912'
    ],
    'Traffic_Intensity': [
        2095, 3199, 17428, 10883, 7592, 11201, 14997, np.nan, 5407, 3415, 2601,
        2579, np.nan, 2746, 1939, np.nan, np.nan, 3479, 5152, 2141, 4107, np.nan
    ]
}

# Create a DataFrame from the traffic data
df_traffic = pd.DataFrame(traffic_data)

# Merge the traffic data with precast_postensioned DataFrame on 'ID_mosta'
precast_postensioned = precast_postensioned.merge(
    df_traffic, on='ID_mosta', how='left')

In [108]:
import folium


def create_bridge_map(df, index):
    # Get bridge data from the DataFrame based on the index
    bridge_data = df.iloc[index]
    latitude = bridge_data['Zemepisná_šírka']
    longitude = bridge_data['Zemepisná_dĺžka']
    bridge_name = bridge_data['Názov_mosta']
    traffic_intensity = bridge_data['Traffic_Intensity']

    # Create a folium map centered on the bridge location
    bridge_map = folium.Map(location=[latitude, longitude], zoom_start=15)

    # Add a marker for the bridge
    folium.Marker(
        location=[latitude, longitude],
        popup=f"<strong>{
            bridge_name}</strong><br>Traffic Intensity: {traffic_intensity}",
        tooltip="Click for more info"
    ).add_to(bridge_map)

    return bridge_map

In [110]:
import os
import xml.etree.ElementTree as ET
import pandas as pd


def add_route_time_distance_to_df(df, folder_path):
    # Initialize columns only if they do not exist
    if 'Clear_Time' not in df.columns:
        df['Clear_Time'] = None
    if 'Clear_Distance' not in df.columns:
        df['Clear_Distance'] = None
    if 'Alt_Time' not in df.columns:
        df['Alt_Time'] = None
    if 'Alt_Distance' not in df.columns:
        df['Alt_Distance'] = None

    # Define namespaces
    namespaces = {
        '': 'http://www.topografix.com/GPX/1/1',
        'gh': 'https://graphhopper.com/public/schema/gpx/1.1'
    }

    # Loop through each bridge in the DataFrame
    for index, row in df.iterrows():
        bridge_id = row['ID_mosta']

        # Find corresponding GPX files for this bridge
        gpx_files = [f for f in os.listdir(folder_path) if f.startswith(
            bridge_id) and f.endswith('.gpx')]

        # Initialize variables for route details
        clear_time, clear_distance = 0, 0
        alt_time, alt_distance = 0, 0

        # Process each GPX file
        for gpx_file in gpx_files:
            file_path = os.path.join(folder_path, gpx_file)
            tree = ET.parse(file_path)
            root = tree.getroot()

            # Initialize total time and distance for this route
            total_time = 0
            total_distance = 0

            # Sum up time and distance from each rtept element
            for rtept in root.findall('.//rtept', namespaces):
                extensions = rtept.find(
                    '{http://www.topografix.com/GPX/1/1}extensions')
                if extensions is not None:
                    time_elem = extensions.find('gh:time', namespaces)
                    distance_elem = extensions.find('gh:distance', namespaces)

                    # Accumulate distance and time if they exist
                    if time_elem is not None:
                        total_time += float(time_elem.text)
                    if distance_elem is not None:
                        total_distance += float(distance_elem.text)

            # Convert time from milliseconds to minutes
            total_time_minutes = (total_time / 1000) / 60

            # Assign route details based on file name
            if '_clear' in gpx_file:
                clear_time, clear_distance = total_time_minutes, total_distance
            elif '_alt' in gpx_file:
                alt_time, alt_distance = total_time_minutes, total_distance

        # Update the DataFrame with the extracted values
        df.at[index, 'Clear_Time'] = clear_time if clear_time != 0 else None
        df.at[index, 'Clear_Distance'] = clear_distance if clear_distance != 0 else None
        df.at[index, 'Alt_Time'] = alt_time if alt_time != 0 else None
        df.at[index, 'Alt_Distance'] = alt_distance if alt_distance != 0 else None

    return df

In [147]:
precast_postensioned.columns

Index(['ID_mosta', 'Názov_mosta', 'Rok_postavenia', 'Dĺžka_premostenia',
       'Zemepisná_šírka', 'Zemepisná_dĺžka', 'n_2012', 'n_2013', 'n_2014',
       'n_2015', 'n_2016', 'n_2017', 'n_2018', 'n_2019', 'n_2020', 'n_2021',
       'n_2022', 'n_2023', 'n_2024', 'Traffic_Intensity', 'Clear_Time',
       'Clear_Distance', 'Alt_Time', 'Alt_Distance'],
      dtype='object')

In [148]:
# Usage: Call the function with the DataFrame and the folder path containing GPX files
precast_postensioned = add_route_time_distance_to_df(
    precast_postensioned, "C:/Users/relia/Documents/GitHub/Bridges/01_SC_BA/DT/Routes/New")

precast_postensioned[["ID_mosta", 'Traffic_Intensity', 'Clear_Time',
                      'Clear_Distance', 'Alt_Time', 'Alt_Distance']]

,ID_mosta,Traffic_Intensity,Clear_Time,Clear_Distance,Alt_Time,Alt_Distance
0,M2798,2095.0,10.401617,11164.469,22.999367,23621.52
1,M1734,3199.0,4.454417,4873.101,16.142367,5262.541
2,M221,17428.0,8.166983,7250.297,12.993033,12346.017
3,M2684,10883.0,5.248583,3480.775,15.43985,9560.415
4,M7488,7592.0,None,None,None,None
5,M5538,11201.0,None,None,None,None
6,M2907,14997.0,None,None,None,None
7,M2273,NaN,8.32075,8328.417,26.464967,23982.639
8,M6631,5407.0,0.8356,445.658,7.895967,4045.962
9,M1016,3415.0,16.04745,17016.009,21.4783,23701.825


In [116]:
import folium
import xml.etree.ElementTree as ET


def create_single_bridge_map(bridge_row, folder_path):
    # Extract bridge information from the row
    bridge_id = bridge_row['ID_mosta']
    bridge_name = bridge_row['Názov_mosta']
    latitude = bridge_row['Zemepisná_šírka']
    longitude = bridge_row['Zemepisná_dĺžka']

    # Retrieve precomputed route info from the DataFrame row
    clear_time = bridge_row['Clear_Time']
    clear_distance = bridge_row['Clear_Distance']
    alt_time = bridge_row['Alt_Time']
    alt_distance = bridge_row['Alt_Distance']

    # Create a Folium map centered on the bridge location
    bridge_map = folium.Map(location=[latitude, longitude], zoom_start=15)

    # Define namespaces for GPX parsing
    namespaces = {
        '': 'http://www.topografix.com/GPX/1/1',
        'gh': 'https://graphhopper.com/public/schema/gpx/1.1'
    }

    # Find corresponding GPX files for this bridge
    gpx_files = [f for f in os.listdir(folder_path) if f.startswith(
        bridge_id) and f.endswith('.gpx')]

    # Process each GPX file to plot the route
    for gpx_file in gpx_files:
        file_path = os.path.join(folder_path, gpx_file)
        tree = ET.parse(file_path)
        root = tree.getroot()

        # Collect route points for plotting
        route_points = [(float(pt.attrib['lat']), float(pt.attrib['lon']))
                        for pt in root.findall('.//trkpt', namespaces)]
        if route_points:
            route_color = 'blue' if '_clear' in gpx_file else 'red'
            folium.PolyLine(
                route_points,
                color=route_color,
                weight=4,
                opacity=0.7,
                popup=f"{gpx_file} Route"
            ).add_to(bridge_map)

    # Prepare popup message with precomputed values
    popup_message = (
        f"<strong>{bridge_name} (ID: {bridge_id})</strong><br>"
        f"<strong>Clear Route:</strong><br>"
        f"Distance: {clear_distance:.2f} m, Time: {clear_time:.2f} min<br>"
        f"<strong>Alt Route:</strong><br>"
        f"Distance: {alt_distance:.2f} m, Time: {alt_time:.2f} min"
    )

    # Add a marker for the bridge with hover (tooltip) and click (popup) info
    folium.Marker(
        location=[latitude, longitude],
        tooltip=f"Bridge ID: {bridge_id}",
        popup=popup_message
    ).add_to(bridge_map)

    return bridge_map

In [146]:
# Usage: Call the function with a specific row and the folder path containing GPX files
bridge_map = create_single_bridge_map(
    precast_postensioned.iloc[-3], "C:/Users/relia/Documents/GitHub/Bridges/01_SC_BA/DT/Routes/New")

# Display the map
bridge_map

In [149]:
import os
import pandas as pd
import gpxpy
import numpy as np

# Define the polynomial function for fuel consumption based on the slope


def get_fuel_consumption(slope):
    return 6 + 0.766 * slope + 0.0646 * slope**2

# Extract route data from a GPX file


def extract_route_data(gpx_file_path):
    with open(gpx_file_path, 'r') as gpx_file:
        gpx = gpxpy.parse(gpx_file)

    latitudes = []
    longitudes = []
    elevations = []
    distances = []
    cumulative_distance = 0
    previous_point = None

    for track in gpx.tracks:
        for segment in track.segments:
            for point in segment.points:
                latitudes.append(point.latitude)
                longitudes.append(point.longitude)
                elevations.append(point.elevation)
                if previous_point is not None:
                    distance = point.distance_2d(previous_point)
                    cumulative_distance += distance
                else:
                    distance = 0
                distances.append(cumulative_distance)
                previous_point = point

    return latitudes, longitudes, elevations, distances

# Process the route to calculate segment details and fuel consumption


def process_route(gpx_file_path):
    # Extract route data
    latitudes, longitudes, elevations, distances = extract_route_data(
        gpx_file_path)

    # Create a DataFrame for route data
    data = {
        'Latitude': latitudes,
        'Longitude': longitudes,
        'Distance (m)': distances,
        'Elevation (m)': elevations
    }

    df_route = pd.DataFrame(data)

    # Calculate segment length and slope for each segment
    segment_lengths = [0]  # Initial segment length is 0 for the starting point
    slopes = [0]           # Initial slope is 0 for the starting point

    for i in range(1, len(df_route)):
        segment_length = df_route['Distance (m)'].iloc[i] - \
            df_route['Distance (m)'].iloc[i - 1]
        elevation_change = df_route['Elevation (m)'].iloc[i] - \
            df_route['Elevation (m)'].iloc[i - 1]
        slope = np.degrees(np.arctan(elevation_change /
                           segment_length)) if segment_length != 0 else 0

        segment_lengths.append(segment_length)
        slopes.append(slope)

    df_route['Segment Length (m)'] = segment_lengths
    df_route['Slope (Degrees)'] = slopes

    # Calculate fuel consumption for each segment
    df_route['Fuel Consumption (liters/100km)'] = df_route['Slope (Degrees)'].apply(
        get_fuel_consumption)

    # Calculate fuel consumption per segment in liters
    df_route['Fuel Consumption per Segment (liters)'] = (
        df_route['Fuel Consumption (liters/100km)'] * df_route['Segment Length (m)']) / 100000

    # Calculate total fuel consumption
    total_fuel_consumption = df_route['Fuel Consumption per Segment (liters)'].sum(
    )

    return total_fuel_consumption


# Initialize columns in the DataFrame for fuel consumption
precast_postensioned['Clear_Fuel_Consumption'] = None
precast_postensioned['Alt_Fuel_Consumption'] = None

# Define folder path for GPX files
folder_path = "C:/Users/relia/Documents/GitHub/Bridges/01_SC_BA/DT/Routes/"

# Loop through each bridge in the DataFrame and calculate fuel consumption for both routes
for index, row in precast_postensioned.iterrows():
    bridge_id = row['ID_mosta']
    clear_file_path = os.path.join(folder_path, f"{bridge_id}_clear.gpx")
    alt_file_path = os.path.join(folder_path, f"{bridge_id}_alt.gpx")

    # Calculate fuel consumption for the clear route if the file exists
    if os.path.exists(clear_file_path):
        clear_fuel_consumption = process_route(clear_file_path)
        precast_postensioned.at[index,
                                'Clear_Fuel_Consumption'] = clear_fuel_consumption

    # Calculate fuel consumption for the alt route if the file exists
    if os.path.exists(alt_file_path):
        alt_fuel_consumption = process_route(alt_file_path)
        precast_postensioned.at[index,
                                'Alt_Fuel_Consumption'] = alt_fuel_consumption

# Display updated DataFrame
precast_postensioned[['ID_mosta',
                      'Clear_Fuel_Consumption', 'Alt_Fuel_Consumption']]

,ID_mosta,Clear_Fuel_Consumption,Alt_Fuel_Consumption
0,M2798,0.63238,1.438329
1,M1734,0.286022,0.314664
2,M221,0.44111,0.747525
3,M2684,0.212844,0.579298
4,M7488,0.140316,0.262178
5,M5538,None,None
6,M2907,None,None
7,M2273,0.506376,1.452177
8,M6631,0.030322,1.099439
9,M1016,1.057591,1.457171


In [140]:
precast_postensioned.columns

Index(['ID_mosta', 'Názov_mosta', 'Rok_postavenia', 'Dĺžka_premostenia',
       'Zemepisná_šírka', 'Zemepisná_dĺžka', 'n_2012', 'n_2013', 'n_2014',
       'n_2015', 'n_2016', 'n_2017', 'n_2018', 'n_2019', 'n_2020', 'n_2021',
       'n_2022', 'n_2023', 'n_2024', 'Traffic_Intensity', 'Clear_Time',
       'Clear_Distance', 'Alt_Time', 'Alt_Distance'],
      dtype='object')

In [153]:
# Calculate the difference in fuel consumption between the clear and alt routes
precast_postensioned['Fuel_Consumption_Difference'] = (
    precast_postensioned['Alt_Fuel_Consumption'] -
    precast_postensioned['Clear_Fuel_Consumption']
)

# Convert the fuel consumption difference to cost (euro) per vehicle
fuel_price_per_liter = 1.516
precast_postensioned['Cost_Per_Vehicle'] = (
    precast_postensioned['Fuel_Consumption_Difference'] * fuel_price_per_liter
)

# Multiply by Traffic Intensity to get the total cost impact
precast_postensioned['Total_Cost'] = (
    precast_postensioned['Cost_Per_Vehicle'] *
    precast_postensioned['Traffic_Intensity']
)

# Display the updated DataFrame with the new columns
precast_postensioned[[
    "ID_mosta", 'Traffic_Intensity', 'Fuel_Consumption_Difference',
    'Cost_Per_Vehicle', 'Total_Cost'
]]

,ID_mosta,Traffic_Intensity,Fuel_Consumption_Difference,Cost_Per_Vehicle,Total_Cost
0,M2798,2095.0,0.805948,1.221818,2559.708372
1,M1734,3199.0,0.028642,0.043421,138.902929
2,M221,17428.0,0.306415,0.464525,8095.745807
3,M2684,10883.0,0.366454,0.555544,6045.989779
4,M7488,7592.0,0.121863,0.184744,1402.575942
5,M5538,11201.0,NaN,NaN,NaN
6,M2907,14997.0,NaN,NaN,NaN
7,M2273,NaN,0.945801,1.433834,NaN
8,M6631,5407.0,1.069117,1.620782,8763.567108
9,M1016,3415.0,0.39958,0.605764,2068.683428


In [163]:
# Set Pandas display option to prevent truncation of long strings
pd.set_option('display.max_colwidth', None)

# Print the DataFrame with bridge IDs and full names
print(precast_postensioned[["ID_mosta", 'Názov_mosta']])

# Reset display option to default if needed later
# pd.reset_option('display.max_colwidth')

   ID_mosta                                                   Názov_mosta
0     M2798                          II_501 - 013A Most na ŽSR - Rohožník
1     M1734                       II_501 - 015 Most nad ŽSR za Rohožníkom
2      M221       II_502 - 006 Most nad polnou cestou a potokom - Sv. Jur
3     M2684                    II_503 - 008 most nad traťou ŽSR - Pezinok
4     M7488                          II_503 - 009 Most na peáži v Pezinku
5     M5538                  II_503 - 022 Most cez potok Malina - Malacky
6     M2907                 II_503 - 022B Most nad ŽSR - Estakáda Malacky
7     M2273                          II_503 - 029 Most cez Zohorský kanál
8     M6631                 II_504 - 004 Most cez potok Gidra - Budmerice
9     M1016             II_590 - 003 Most nad diaľnicou D2 - za Malackami
10    M3641          II_590 - 004 Most cez potok Rudava - pred Studienkou
11    M3844         III_002032 - 002 Nadjazd nad D2 za obcou Veľké Leváre
12    M7647  III_002034 - 002 Most cez

In [176]:
# Usage: Call the function with a specific row and the folder path containing GPX files
bridge_map = create_single_bridge_map(
    precast_postensioned.iloc[2], "C:/Users/relia/Documents/GitHub/Bridges/01_SC_BA/DT/Routes/New")

# Display the map
bridge_map

In [175]:
precast_postensioned.iloc[2]

ID_mosta                                                                          M221
Názov_mosta                    II_502 - 006 Most nad polnou cestou a potokom - Sv. Jur
Rok_postavenia                                                                    1983
Dĺžka_premostenia                                                                 11.0
Zemepisná_šírka                                                              48.241013
Zemepisná_dĺžka                                                              17.203997
n_2012                                                                               4
n_2013                                                                               4
n_2014                                                                               4
n_2015                                                                               4
n_2016                                                                               5
n_2017                                     

In [177]:
precast_postensioned["Total_Cost"].mean()

np.float64(2763.180436460018)

In [171]:
precast_postensioned["Diff_time"] = (precast_postensioned["Alt_Time"] 
                                     - precast_postensioned["Clear_Time"])

precast_postensioned["Diff_dist"] = (precast_postensioned["Alt_Distance"]
                                     - precast_postensioned["Clear_Distance"]) / 1000

In [164]:
# Define the path and filename for the Excel file
output_path = "C:/Users/relia/Documents/Precast_Postensioned_Bridges.xlsx"

# Export the DataFrame to an Excel file
precast_postensioned.to_excel(output_path, index=False)

print(f"DataFrame exported successfully to {output_path}")

DataFrame exported successfully to C:/Users/relia/Documents/Precast_Postensioned_Bridges.xlsx
